In [2]:
#练习线性回归之梯度下降，Lasso回归，岭回归，理解原理是核心
import time

from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

fetch_data = fetch_california_housing(data_home='data')#获取数据集
print("获取特征值")
print(fetch_data.data.shape)#获取特征值维度
print(fetch_data.data[0])#获取特征值
print("获取目标值")
print(fetch_data.target)#获取目标值
print(fetch_data.feature_names)#获取特征名

获取特征值
(20640, 8)
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
获取目标值
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [10]:
fetch_data.target.shape#获取目标值维度

(20640,)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(fetch_data.data, fetch_data.target, test_size=0.25, random_state=1)#分割数据集到训练集和测试集
print(x_train.shape)#训练集特征值维度
std_x = StandardScaler()#标准化
x_train = std_x.fit_transform(x_train)#标准化训练集
x_test = std_x.transform(x_test)#标准化测试集
print(y_train.shape)
std_y = StandardScaler()#标准化目标值
y_train = std_y.fit_transform(y_train.reshape(-1, 1))#标准化目标值
print(y_train.shape)#重塑后的目标值维度

(15480, 8)
(15480,)
(15480, 1)


In [12]:
temp = y_train.reshape(-1, 1)#重塑目标值维度
print(temp.shape)#重塑后的目标值维度

(15480, 1)


In [13]:
test1 = np.array([1,2,3])#创建数组
print(test1.shape)#数组维度
print(test1.reshape(-1, 1))#重塑数组维度

(3,)
[[1]
 [2]
 [3]]


In [27]:
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, LogisticRegression, Lasso
import os
lr = LinearRegression()#线性回归
lr.fit(x_train, y_train)#训练模型
print("回归系数:")
print(lr.coef_)#回归系数
y_predict = lr.predict(x_test)#预测结果
os.unlink('./tmp/test.pkl') # 删除之前的模型文件
joblib.dump(lr, "./tmp/test.pkl")
print("正规方程测试集里面每个房子的预测价格：", y_predict[0:10])#打印预测结果
print("正规方程的均方误差：", mean_squared_error(y_test, y_predict))#打印均方误差

回归系数:
[ 0.83167028  0.12159502 -0.26758589  0.30983997 -0.00518054 -0.04040421
 -0.90736902 -0.88212727]
正规方程测试集里面每个房子的预测价格： [2.12391852 0.93825754 2.7088455  1.70873764 2.82954754 3.50376456
 3.0147162  1.62781292 1.74317518 2.01897806]
正规方程的均方误差： 0.5356532845422556


In [29]:
lr = LinearRegression()#线性回归
lr.fit(x_train, y_train)#训练模型
print("回归系数:")
print(lr.coef_)#回归系数
y_predict = lr.predict(x_test)#预测结果
y_lr_predict = std_y.inverse_transform(y_predict)#反标准化预测结果
os.unlink('./tmp/test.pkl') # 删除之前的模型文件
joblib.dump(lr, "./tmp/test.pkl")
print("正规方程测试集里面每个房子的预测价格：", y_lr_predict[0:10])
print("正规方程的均方误差：", mean_squared_error(y_test, y_lr_predict))#打印均方误差

回归系数:
[[ 0.71942632  0.10518431 -0.23147194  0.26802332 -0.00448136 -0.03495117
  -0.7849086  -0.76307353]]
正规方程测试集里面每个房子的预测价格： [[2.12391852]
 [0.93825754]
 [2.7088455 ]
 [1.70873764]
 [2.82954754]
 [3.50376456]
 [3.0147162 ]
 [1.62781292]
 [1.74317518]
 [2.01897806]]
正规方程的均方误差： 0.5356532845422556


In [30]:
model = joblib.load("./tmp/test.pkl")#加载模型
y_predict = model.predict(x_test)#预测结果
print("保存的y标准化后的模型预测的结果：", std_y.inverse_transform(y_predict)[0:10])#打印预测结果
print("正规方程inverse后的均方误差：", mean_squared_error(y_test,
                                               std_y.inverse_transform(y_predict)))#打印均方误差

保存的y标准化后的模型预测的结果： [[2.12391852]
 [0.93825754]
 [2.7088455 ]
 [1.70873764]
 [2.82954754]
 [3.50376456]
 [3.0147162 ]
 [1.62781292]
 [1.74317518]
 [2.01897806]]
正规方程inverse后的均方误差： 0.5356532845422556


In [31]:
y_true = [3, -0.5, 2, 7]#真实值
y_pred = [2.5, 0.0, 2, 8]#预测值
mean_squared_error(y_true, y_pred)#计算均方误差

0.375

In [53]:
#线性回归之梯度下降
sgd = SGDRegressor(eta0=0.01,penalty='l2', max_iter=1000)#创建SGDRegressor对象
sgd.fit(x_train, y_train)#训练模型
print('梯度下降的回归系数', sgd.coef_)#打印回归系数
y_predict = sgd.predict(x_test)#预测结果
print("梯度下降的均方误差：", mean_squared_error(y_test, y_predict))#打印均方误差

梯度下降的回归系数 [ 0.70944479  0.09336909 -0.20565056  0.23581556 -0.02097679 -0.12699846
 -0.77758674 -0.76332109]
梯度下降的均方误差： 4.75094456098578


C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
#Lasso回归
print(x_train.shape)
print(y_train.shape)
ls = Lasso(alpha=0.001)#创建Lasso对象
ls.fit(x_train, y_train)#训练模型
print('Lasso的回归系数', ls.coef_)#打印回归系数
print(ls.predict(x_test).shape)#预测结果维度
y_predict = ls.predict(x_test)#预测结果
print("Lasso的均方误差：", mean_squared_error(y_test, y_predict))#打印均方误差

(15480, 8)
(15480, 1)
Lasso的回归系数 [ 0.71431792  0.10613811 -0.21758465  0.25415162 -0.00311065 -0.03403136
 -0.77399969 -0.75154125]
(5160,)
Lasso的均方误差： 4.770944530791325


In [59]:
#岭回归
rd = Ridge(alpha=0.001)#创建Ridge对象
rd.fit(x_train, y_train)#训练模型
print('岭回归的回归系数', rd.coef_)#打印回归系数
print(rd.predict(x_test).shape)#预测结果维度
y_predict = rd.predict(x_test)#预测结果
print("岭回归的均方误差：", mean_squared_error(y_test, y_predict))#打印均方误差

岭回归的回归系数 [ 0.7194263   0.10518438 -0.23147179  0.26802312 -0.00448134 -0.03495118
 -0.78490787 -0.7630728 ]
(5160,)
岭回归的均方误差： 4.7705005245159136
